In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import random
from lxml.html import fromstring
from itertools import cycle
import time

In [2]:
# user agents to choose from
user_agents_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
]

In [3]:
def xml_parser():
    """returns xml link to wine_review_sitemap"""
    wine_review_sitemaps = []
    url = "https://www.winemag.com/sitemap_index.xml"    
    r= requests.get(url, headers = {"User-Agent": random.choice(user_agents_list)})
    soup = BeautifulSoup(r.text)
    locs = soup.find_all("loc")
    for loc in locs:
        splited = loc.text.split("/")
        if "wine_review-sitemap" in splited[-1]:
            wine_review_sitemaps.append(loc.text)
    return wine_review_sitemaps

In [4]:
xmls = xml_parser()

In [5]:
def wine_review_xml_parser(xml_l):
    """Extract url to each wine and return in a list form"""
    wine_urls = []
    pbar = tqdm(total = len(xml_l))
    for xml in xml_l:
        time.sleep(5)
        r = requests.get(xml, headers = {"User-Agent":random.choice(user_agents_list)})
        soup = BeautifulSoup(r.text)
        locs = soup.find_all("loc")
        for loc in locs:
            wine_urls.append(loc.text)
        pbar.update(1)
    return wine_urls

In [16]:
wine_urls = wine_review_xml_parser(xmls)

In [17]:
# save urls
with open("data/wine_urls.txt", "w") as f:
    for wine_url in wine_urls:
        f.write(wine_url+"\n")
    f.close()

In [13]:
def get_wine_urls():
    """Reads urls to each wines"""
    urls = []
    with open("data/wine_urls.txt", "r") as f:
        lines= f.readlines()
        for line in lines:
            urls.append(line.replace("\n",""))
    return urls

In [21]:
def wine_info_parser(soup):
    """Extract wanted information from each page"""
    wine_info = {}
    wine_name = soup.h1.text
    wine_info["wine_name"] = wine_name
    
    if soup.find("div", class_="badge") != None:
        badge = soup.find("div", class_="badge").span.text
        wine_info["badge"] = badge
    desc = soup.find("p", class_="description")
    
    if soup.find("span", class_="taster-area") == None:
        taster = None
    else:
        taster = soup.find("span", class_="taster-area").text

    if taster !=None:
        desc_txt = desc.text.replace(taster, "").strip()
    else:
        desc_txt = desc.text.strip()
    wine_info["taster"] = taster
    wine_info["desc"] = desc_txt
    
    wine_info = primary_info_parser(soup, wine_info)
    wine_info = secondary_info_parser(soup, wine_info)
    return wine_info
    
    
def primary_info_parser(soup, d):
    """parse primary info section"""
    lis = soup.find("ul", class_= "primary-info").find_all("li")
    for li in lis:
        divs = li.find_all("div")
        d[divs[0].text] = divs[1].text
    return d

def secondary_info_parser(soup, d):
    """parse secondary info section"""
    lis = soup.find("ul", class_= "secondary-info").find_all("li")
    for li in lis:
        divs = li.find_all("div")
        d[divs[0].text] = divs[1].text
    return d
def collect_wine_details(sampled_urls):
    """requesting information to each url"""
    wine_detail_list = []
    pbar = tqdm(total = len(sampled_urls), desc="Samples")
    for i,sample in enumerate(sampled_urls):
        print(sample)
        r = requests.get(sample,headers= {"User-Agent":random.choice(user_agents_list)})
        time.sleep(4+random.random())
        if r.status_code != 200:
            print(r.status_code, sample)
        else:
            wine_detail_soup = BeautifulSoup(r.text)
            wine_detail_list.append(wine_info_parser(wine_detail_soup))
        pbar.update(1)
        
        
    return wine_detail_list

def newline_strip(x):
    """returns string with cleaned format: used for formatting all columns"""
    if str(x) == "nan" or x == None:
        pass
    else:
        return x.strip("\n")

def Price_col_formatter(x):
    """returns string with cleaned format: used for formatting price column"""
    if str(x) == "nan" or x ==None:
        pass
    else:
        return x.split(",")[0]

In [142]:
wine_urls = get_wine_urls() #302684 different wines
# randomly choose 1000 wines
wine_sample_urls = random.sample(wine_urls,1000)
# scrape wine information
# REMOVE COMMENT ONLY IF SCRAPING (TAKES LONG)
#wine_details_list = collect_wine_details(wine_sample_urls)
df = pd.DataFrame(wine_details_list)
# cleaning
clean_cols = [c.strip("\n") for c in df.columns]
df.columns = clean_cols

for c in df.columns:
    df[c] = df[c].apply(newline_strip)
    
df["Price"] = df["Price"].apply(Price_col_formatter)

302684


In [132]:
# save data frame
df.to_csv("data/samples/1000wine_samples.csv")

In [19]:
# scraping wine data this time choosing 2k
wine_urls = get_wine_urls()
wine_2k_sample_urls = random.sample(wine_urls, 2000) # going to be the meta data
wine_2k_details = collect_wine_details(wine_2k_sample_urls)
meta_df = pd.DataFrame(wine_2k_details)

https://www.winemag.com/buying-guide/mcgregor-2007-rob-roy-red-red-finger-lakes/
https://www.winemag.com/buying-guide/nectar-of-the-dogs-2017-3-dogs-white-santa-barbara-county-white-blend/
https://www.winemag.com/buying-guide/cantina-bergamasca-2010-pinot-bianco-lombardy/
https://www.winemag.com/buying-guide/belvedere-2000-chardonnay-russian-river-valley/
https://www.winemag.com/buying-guide/jermann-2018-sauvignon-blanc-venezia-giulia/
https://www.winemag.com/buying-guide/hickinbotham-2013-brooks-road-shiraz-mclaren-vale/
https://www.winemag.com/buying-guide/pontecilla-2009-old-vines-tempranillo-calatayud-129528/
https://www.winemag.com/buying-guide/manzone-2011-fraschin-barolo/
https://www.winemag.com/buying-guide/hosmer-2014-cabernet-franc-cayuga-lake/

https://www.winemag.com/buying-guide/cave-de-turckheim-2012-hengst-grand-cru-riesling-alsace/
https://www.winemag.com/buying-guide/b-r-cohn-2009-silver-label-cabernet-sauvignon-north-coast/
https://www.winemag.com/buying-guide/chateau

https://www.winemag.com/buying-guide/tamber-bey-2006-deux-chevaux-vineyard-chardonnay-yountville/
https://www.winemag.com/buying-guide/cameron-hughes-2007-lot-96-cabernet-sauvignon-stellenbosch-96669/
https://www.winemag.com/buying-guide/chronicle-2009-savoy-vineyard-pinot-noir-anderson-valley/
https://www.winemag.com/buying-guide/graci-2010-quota-600-etna/
https://www.winemag.com/buying-guide/domaine-rene-bouvier-2018-grand-cru-echezeaux/
https://www.winemag.com/buying-guide/fowles-wine-2016-farm-to-table-cabernet-sauvignon-victoria/
https://www.winemag.com/buying-guide/sula-nv-brut-tropicale-methode-traditionnelle-sparkling-nashik/
https://www.winemag.com/buying-guide/sierra-cantabria-2014-crianza-rioja/
https://www.winemag.com/buying-guide/pierre-sparr-nv-brut-reserve-sparkling-cremant-dalsace-90231/
https://www.winemag.com/buying-guide/poggio-antico-2005-madre-red-toscana-80371/
https://www.winemag.com/buying-guide/livio-felluga-2010-friulano-colli-orientali-del-friuli/
https://www

https://www.winemag.com/buying-guide/joel-gott-2007-mohr-fry-ranches-zinfandel-lodi/
https://www.winemag.com/buying-guide/wolf-blass-2003-gold-label-sauvignon-blanc-adelaide-hills-32216/
https://www.winemag.com/buying-guide/laetitia-2009-les-galets-pinot-noir-arroyo-grande-valley/
https://www.winemag.com/buying-guide/vite-colte-2016-la-casa-in-collina-barbaresco/
https://www.winemag.com/buying-guide/podere-canalino-2004-brunello-di-montalcino/
https://www.winemag.com/buying-guide/henri-de-villamont-2015-clos-des-guettes-premier-cru-savigny-les-beaune/
https://www.winemag.com/buying-guide/laboure-roi-2007-santenots-premier-cru-volnay/
https://www.winemag.com/buying-guide/puramun-2016-reserva-malbec-uco-valley/
https://www.winemag.com/buying-guide/giribaldi-1998-barbaresco/
https://www.winemag.com/buying-guide/vinaceous-2017-raconteur-cabernet-sauvignon-margaret-river/
https://www.winemag.com/buying-guide/cantine-russo-2010-rampante-rosso-etna/
https://www.winemag.com/buying-guide/willam

https://www.winemag.com/buying-guide/hartenberg-2013-cape-winemakers-guild-cwg-auction-shiraz-stellenbosch/
https://www.winemag.com/buying-guide/lynmar-2013-quail-hill-vineyard-summit-pinot-noir-russian-river-valley/
https://www.winemag.com/buying-guide/chateau-de-lavernette-2014-les-vignes-de-la-roche-beaujolais-blanc/
https://www.winemag.com/buying-guide/odfjell-2004-orzada-carignan-maule-valley-84611/
https://www.winemag.com/buying-guide/chasing-venus-2009-pinot-gris-marlborough/
https://www.winemag.com/buying-guide/whitcraft-2018-stolpman-vineyard-grenache-ballard-canyon/
https://www.winemag.com/buying-guide/flanagan-2014-serenity-way-red-bennett-valley/
https://www.winemag.com/buying-guide/damilano-2004-le-cinque-vigne-barolo/
https://www.winemag.com/buying-guide/kentia-2012-albarino-rias-baixas-180226/
https://www.winemag.com/buying-guide/novella-2009-synergy-blanc-white-paso-robles/
https://www.winemag.com/buying-guide/lang-reed-2000-wild-hare-cabernet-franc-rutherford/
https://

https://www.winemag.com/buying-guide/domaine-weinbach-2017-cuvee-ste-catherine-schlossberg-grand-cru-riesling-alsace/
https://www.winemag.com/buying-guide/mendocino-hill-1999-syrah-mendocino-county/
https://www.winemag.com/buying-guide/domaine-serene-2007-clos-du-soleil-chardonnay-dundee-hills/
https://www.winemag.com/buying-guide/hopkins-vineyard-2006-estate-bottled-ice-wine-vidal-blanc-western-connecticut-highlands/
https://www.winemag.com/buying-guide/dominique-portet-2002-cabernet-sauvignon-heathcote-44600/
https://www.winemag.com/buying-guide/nittnaus-hans-und-christine-2005-turn-me-red-zweigelt-burgenland/
https://www.winemag.com/buying-guide/ziata-2017-sauvignon-blanc-napa-valley-300856/
https://www.winemag.com/buying-guide/de-loach-2010-vineyard-designate-pinot-noir-green-valley/
https://www.winemag.com/buying-guide/prunotto-2008-barbaresco/
https://www.winemag.com/buying-guide/miali-2009-ichore-red-salento-164665/
https://www.winemag.com/buying-guide/fife-2000-redhead-rose-red

...

https://www.winemag.com/buying-guide/ra-pfaffl-2014-goldjoch-reserve-gruner-veltliner-weinviertel/
https://www.winemag.com/buying-guide/secret-spot-wines-2014-valpacos-red-tras-os-montes/
https://www.winemag.com/buying-guide/bruno-cormerais-2012-chambaudiere-sur-lie-muscadet-sevre-et-maine/
https://www.winemag.com/buying-guide/santadi-2006-terre-brune-superiore-carignano-del-sulcis/
https://www.winemag.com/buying-guide/howell-mountain-vineyards-2002-beatty-ranch-zinfandel-howell-mountain/
https://www.winemag.com/buying-guide/poggi-wines-2014-cabernet-sauvignon-diamond-mountain-district/
https://www.winemag.com/buying-guide/gramercy-2018-viognier-columbia-valley-wa/
https://www.winemag.com/buying-guide/robert-hall-2005-chardonnay-paso-robles/
https://www.winemag.com/buying-guide/criterion-2013-reserva-malbec-mendoza/
https://www.winemag.com/buying-guide/casa-nuestra-2007-riesling-napa-valley-83002/
https://www.winemag.com/buying-guide/vignobles-bulliat-2016-beaujolais-villages/
https://

https://www.winemag.com/buying-guide/leeuwin-estate-2015-siblings-sauvignon-blanc-semillon-margaret-river/
https://www.winemag.com/buying-guide/coria-2016-mary-sharon-estate-pinot-noir-willamette-valley/
https://www.winemag.com/buying-guide/midnight-2017-nebula-cabernet-sauvignon-paso-robles/
https://www.winemag.com/buying-guide/capanne-ricci-2008-brunello-di-montalcino/
https://www.winemag.com/buying-guide/two-angels-2016-petite-sirah-red-hills/
https://www.winemag.com/buying-guide/courtney-benham-2009-petite-sirah-alexander-valley/
https://www.winemag.com/buying-guide/aether-wine-company-2014-sierra-madre-vineyard-chardonnay-santa-maria-valley/
https://www.winemag.com/buying-guide/cellars-33-2016-the-betty-white-lodi/
https://www.winemag.com/buying-guide/cavicchioli-2017-vigna-del-cristo-lambrusco-di-sorbara/
https://www.winemag.com/buying-guide/yvon-mau-2006-premius-bordeaux/
https://www.winemag.com/buying-guide/ascheri-2016-ascheri-barolo/
https://www.winemag.com/buying-guide/cave-

https://www.winemag.com/buying-guide/pepperwood-grove-2000-pinot-noir-california/
https://www.winemag.com/buying-guide/wagner-2016-unoaked-chardonnay-finger-lakes/
https://www.winemag.com/buying-guide/bodegas-bilbainas-2018-vina-pomal-rose-rioja/
https://www.winemag.com/buying-guide/chateau-bellevue-2012-cuvee-vieilles-vignes-castillon-cotes-de-bordeaux/
https://www.winemag.com/buying-guide/teso-la-monja-2014-alabaster-toro/
https://www.winemag.com/buying-guide/longwood-1999-shiraz-mclaren-vale/
https://www.winemag.com/buying-guide/william-fevre-2005-beauroy-premier-cru-chablis/
https://www.winemag.com/buying-guide/j-rickards-2011-1908-brignole-vineyard-old-vine-zinfandel-alexander-valley/
https://www.winemag.com/buying-guide/pighin-2010-pinot-grigio-collio/
https://www.winemag.com/buying-guide/principe-de-viana-2011-edicion-limitada-red-navarra/
https://www.winemag.com/buying-guide/hopler-2015-pinot-noir-burgenland/
https://www.winemag.com/buying-guide/handcraft-2014-artisan-collectio

https://www.winemag.com/buying-guide/tabor-2014-zohar-adama-ii-white-galilee/
https://www.winemag.com/buying-guide/frey-nv-organic-red-california/
https://www.winemag.com/buying-guide/cooks-nv-white-zinfandel-rose-california/
https://www.winemag.com/buying-guide/charles-sohler-et-ses-fils-nv-blanc-de-blancs-sparkling-cremant-dalsace/
https://www.winemag.com/buying-guide/domaine-le-bout-du-lieu-2007-orbe-noir-malbec-cahors/
https://www.winemag.com/buying-guide/tsantali-2002-ambelonas-white-halkidiki-35888/
https://www.winemag.com/buying-guide/brunelli-2011-pariondo-valpolicella-classico-superiore-ripasso/
https://www.winemag.com/buying-guide/villa-carra-2001-pinot-grigio-friuli-grave/
https://www.winemag.com/buying-guide/nederburg-2002-edelrood-red-western-cape/
https://www.winemag.com/buying-guide/flam-2016-reserve-syrah-galilee/
https://www.winemag.com/buying-guide/feudi-di-san-gregorio-2008-privilegio-white-campania/
https://www.winemag.com/buying-guide/b-cellars-2016-dutton-ranch-ch

NameError: name 'clean_cols' is not defined

In [22]:
# clean formatting
meta_clean_cols = [c.strip("\n") for c in meta_df.columns]
meta_df.columns = meta_clean_cols

for col in meta_df.columns:
    meta_df[col] = meta_df[col].apply(newline_strip)
    
meta_df["Price"] = meta_df["Price"].apply(Price_col_formatter)

# save df
meta_df.to_csv("data/samples/wine_2k_meta.csv")